# OpenStreetMap Point Of Interest Graph

Import points of interest from the [Daylight Earth Table OpenStreetMap](https://daylightmap.org/earth/) distribution into a Neo4j database.

![](img/osm_poi_graph.png)

In [54]:
import pandas
from neo4j import GraphDatabase
import time
import json

## Load Daylight Earth Table Parquet Into Pandas DataFrame

In [3]:
# See data/README.md - the Daylight Earth Table is published as Parquet files in S3
# We will use Pandas to process these Parquet files and work with a DataFrame

df = pandas.read_parquet('./data/poi')

In [4]:
df

,geometry_id,class,subclass,metadata,original_source_tags,names,quadkey,wkt
0,n6124710136@2,commercial,fabric,"{""is_area"":false,""quadkey"":""120223012123112""}","{""addr:housenumber"":""11"",""addr:street"":""Corso ...","{""local"":""Tessilarte 81""}",120223012123112,POINT (7.5458496 44.3872338)
1,w919779614@1,parking,parking_space,"{""amenity"":""parking_space"",""is_area"":true,""qua...","{""amenity"":""parking_space"",""capacity"":""1"",""par...",{},031311121112031,POINT (-1.5464407275117145 54.91039713176086)
2,n289353412@2,recreation,playground,"{""is_area"":false,""quadkey"":""120221100321321""}","{""access"":""yes"",""leisure"":""playground""}",{},120221100321321,POINT (8.934112 48.527624)
3,n3241289585@2,tourism,viewpoint,"{""is_area"":false,""quadkey"":""030232112321013""}","{""tourism"":""viewpoint""}","{""local"":""Wanika Falls""}",030232112321013,POINT (-74.055717 44.1986092)
4,w407412726@1,commercial,retail,"{""building"":""retail"",""height_m"":5.5,""is_buildi...","{""building"":""retail"",""building:units"":""2"",""ele...",{},023012311310033,POINT (-118.2655498069126 33.98113907737265)
...,...,...,...,...,...,...,...,...
38777945,w375652631@1,parking,parking,"{""amenity"":""parking"",""is_area"":true,""quadkey"":...","{""amenity"":""parking"",""parking"":""surface""}",{},120223100310113,POINT (9.04611527653656 44.83077187234537)
38777946,w186376379@1,parking,parking,"{""amenity"":""parking"",""is_area"":true,""quadkey"":...","{""amenity"":""parking""}",{},121200312210212,POINT (49.41902012598777 53.50683910903766)
38777947,w73784956@2,recreation,swimming_pool,"{""is_area"":true,""quadkey"":""120222112310202""}","{""leisure"":""swimming_pool"",""source"":""cadastre-...",{},120222112310202,POINT (4.7539977549157975 44.29432166646483)
38777948,n4842464713@5,food,restaurant,"{""amenity"":""restaurant"",""is_area"":false,""quadk...","{""amenity"":""restaurant"",""check_date:opening_ho...","{""local"":""Zum Bären""}",120221120230310,POINT (8.6835075 47.5999337)


In [11]:
df.head()

,geometry_id,class,subclass,metadata,original_source_tags,names,quadkey,wkt
0,n6124710136@2,commercial,fabric,"{""is_area"":false,""quadkey"":""120223012123112""}","{""addr:housenumber"":""11"",""addr:street"":""Corso ...","{""local"":""Tessilarte 81""}",120223012123112,POINT (7.5458496 44.3872338)
1,w919779614@1,parking,parking_space,"{""amenity"":""parking_space"",""is_area"":true,""qua...","{""amenity"":""parking_space"",""capacity"":""1"",""par...",{},031311121112031,POINT (-1.5464407275117145 54.91039713176086)
2,n289353412@2,recreation,playground,"{""is_area"":false,""quadkey"":""120221100321321""}","{""access"":""yes"",""leisure"":""playground""}",{},120221100321321,POINT (8.934112 48.527624)
3,n3241289585@2,tourism,viewpoint,"{""is_area"":false,""quadkey"":""030232112321013""}","{""tourism"":""viewpoint""}","{""local"":""Wanika Falls""}",030232112321013,POINT (-74.055717 44.1986092)
4,w407412726@1,commercial,retail,"{""building"":""retail"",""height_m"":5.5,""is_buildi...","{""building"":""retail"",""building:units"":""2"",""ele...",{},023012311310033,POINT (-118.2655498069126 33.98113907737265)


## DataFrame Clean Up

In [13]:
# To start off let's filter for points of interest that have a name.
dffilter = df['names'].str.contains('local')
tempdf = df[dffilter]

In [36]:
# Next, let's convert some of the JSON strings stored in our DataFrame into Python dicts

def tryconvert(value, default):
    try:
        return json.loads(value)
    except Exception as e:
        pass
    return default

tempdf['names'] = tempdf['names'].apply(lambda x: tryconvert(x, {'name': "NA"}))

tempdf['original_tags_dict'] = tempdf['original_source_tags'].apply(lambda x: tryconvert(x, {}))


In [ ]:
# And let's convert out Point geometry in WKT to a list of latitude/longitude coordinates
tempdf['point'] = tempdf['wkt'].apply(lambda x: x.replace('POINT (', "").replace(')', '').split())

In [103]:
tempdf.head()

,geometry_id,class,subclass,metadata,original_source_tags,names,quadkey,wkt,point,original_tags_dict
0,n6124710136@2,commercial,fabric,"{""is_area"":false,""quadkey"":""120223012123112""}","{""addr:housenumber"":""11"",""addr:street"":""Corso ...",{'local': 'Tessilarte 81'},120223012123112,POINT (7.5458496 44.3872338),"[7.5458496, 44.3872338]","{'addr:housenumber': '11', 'addr:street': 'Cor..."
3,n3241289585@2,tourism,viewpoint,"{""is_area"":false,""quadkey"":""030232112321013""}","{""tourism"":""viewpoint""}",{'local': 'Wanika Falls'},030232112321013,POINT (-74.055717 44.1986092),"[-74.055717, 44.1986092]",{'tourism': 'viewpoint'}
6,n4740980225@1,other,poi,"{""is_area"":false,""quadkey"":""132221130201020""}","{""phone"":""+6677270670"",""tourism"":""hotel""}",{'local': 'Centra Coconut beach resort samui'},132221130201020,POINT (99.9374697 9.4192596),"[99.9374697, 9.4192596]","{'phone': '+6677270670', 'tourism': 'hotel'}"
9,n7106398577@1,medical,hospital,"{""amenity"":""hospital"",""is_area"":false,""quadkey...","{""addr:district"":""Ludhiana"",""addr:full"":""Line ...","{'local': 'Raman Hospital, Ludhiana'}",123121013312320,POINT (75.8115623 30.9155611),"[75.8115623, 30.9155611]","{'addr:district': 'Ludhiana', 'addr:full': 'Li..."
10,n1952358265@4,food,restaurant,"{""amenity"":""restaurant"",""is_area"":false,""quadk...","{""addr:city"":""Roma"",""addr:country"":""IT"",""addr:...",{'local': 'Pizzeria L'Economica'},120232221130010,POINT (12.5128048 41.8974497),"[12.5128048, 41.8974497]","{'addr:city': 'Roma', 'addr:country': 'IT', 'a..."


## Neo4j Import

See https://towardsdatascience.com/create-a-graph-database-in-neo4j-using-python-4172d40f89c4

In [96]:
# Let's create a class to abstract the Neo4j Python driver API

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exeception as e:
            print("Failed to create the driver:", e)
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
    
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            # TODO: convert to transaction function / 5.0 driver API
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

conn = Neo4jConnection(uri="neo4j://localhost", user="neo4j", pwd="letmein")

In [99]:
# Here we define a function to split our DataFrame into batches for import

def insert_data(query, rows, batch_size=10000):
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):
        res = conn.query(query, parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')}, db="osmpois")
        total += res[0]['total']
        batch += 1
        result = {"total": total, "batches": batch, "time": time.time()-start}
        print(result)
    return result

In [100]:
# We'll define a Cypher Query to create data in Neo4j

def insert_pois(rows, batch_size=10000):
    print(rows)
    query = '''
    UNWIND $rows AS row
    CREATE (p:PointOfInterest {name: row.names.local})
    CREATE (g:Geometry)
    SET g.location = point({latitude: toFloat(row.point[1]), longitude: toFloat(row.point[0]) })
    CREATE (g)<-[:HAS_GEOMETRY]-(p)
    SET g:Point
    CREATE (t:Tags)
    SET t += row.original_tags_dict
    CREATE (p)-[:HAS_TAGS]->(t)
    WITH *
    CALL apoc.create.addLabels(p, [row.class, row.subclass]) YIELD node
    RETURN COUNT(*) AS total
    '''
    return insert_data(query, rows, batch_size)

In [ ]:
insert_pois(tempdf)